# Warm up
Let's start with some warm up exercises to get you familiar with the database and do some SQL querying.
Make sure to have downloaded the NBA dataset form Kaggle as described in the README.

In [ ]:
import sqlite3 as sql
import pandas as pd
from pathlib import Path

## TODO: Use pathlib to get the path to the data directory. Path(__file__) won't work in a Jupyter notebook.
DATA_PATH = Path.cwd() / 'data' / 'nba.sqlite'

## Create a SQLite connection and run some SQL queries

In [ ]:
con = sql.connect(DATA_PATH)  # connect to the database
query = 'SELECT * FROM game LIMIT 10'  # write a query that selects the first 10 rows from the game table
top_10_game = pd.read_sql(query, con)  # read the query into a pandas dataframe
top_10_game.head()  # print the dataframe

In [ ]:
# let's look at the schema of the database
# TODO: sqlite_master is the name of the table you want to inspeact: Replace TABLE_NAME with the name of the table you want to inspect
query = """
    SELECT *
    FROM sqlite_master;
"""

pd.read_sql(query, con).sort_values('name')

In [ ]:
# let's see all the tables we have in the database
# TODO: Filter for tables only
query = """
    SELECT name
    FROM sqlite_master
    WHERE type='table';
"""
pd.read_sql(query, con)

In [ ]:
# let's have a look at the columns in the game table
query = """
    PRAGMA table_info(draft_combine_stats)
"""
pd.read_sql(query, con)

In [ ]:
# let's find the earliest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date
    LIMIT 1
"""

print(pd.read_sql(query, con))

# let's find the latest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date DESC
    LIMIT 1
"""

print(pd.read_sql(query, con))


In [ ]:
# I am interested in finding out whether players have always had the same jersey numbers, or if they worn different jerseys. 
# As we can see in the data, 108 players have switched their jersey number at least once in their careers
query = """
    SELECT first_name, last_name, GROUP_CONCAT(DISTINCT jersey_num) AS jersey_numbers
    FROM officials
    GROUP BY first_name, last_name
    HAVING COUNT(DISTINCT jersey_num) > 1
    ORDER BY last_name, first_name;
"""

pd.read_sql(query, con)

In [ ]:
# I want to find the oldest nba team
# My query suggests Boston Celtic is the oldest team founded in 1946
query = """
    SELECT full_name, year_founded
    FROM team
    ORDER BY year_founded
    LIMIT 1
"""

print(pd.read_sql(query, con))

In [ ]:
# I want to find out which players went to the same school. For example, both Chris Dudley, Tony Lavelli and Miye Oni are from Yale
query = """ 
    SELECT school, GROUP_CONCAT(first_name || ' ' || last_name) AS players
    FROM common_player_info
    GROUP BY school
    HAVING COUNT(*) > 1
    ORDER BY school
"""
school_data = pd.read_sql(query,con)
school_data.head(20)


In [ ]:
# I want to see if anyone went to Harvard
school_data[school_data["school"] == 'Harvard']

In [ ]:
# I want to figure out who the youngest player is in nba: Jalen Duren who was born on November 18, 2003
query = """ 
    SELECT first_name, last_name, birthdate
    FROM common_player_info
    ORDER BY birthdate DESC
    LIMIT 1
"""

pd.read_sql(query,con)

In [ ]:
# I am also curious whether birth dates of NBA players follow any specific distribution
query = """ 
    SELECT birthdate
    FROM common_player_info
"""

data  = pd.read_sql(query, con)

data["birthdate"] = pd.to_datetime(data["birthdate"])
data.hist(bins=20)
